# Bitcoin Price Prediction Time-Series Project

### Data Head

| Timestamp           |   Open |   High |    Low |   Close |   Volume_(BTC) |   Volume_(Currency) |   Weighted_Price |
|:--------------------|-------:|-------:|-------:|--------:|---------------:|--------------------:|-----------------:|
| 2011-12-31 07:52:00 |   4.39 |   4.39 |   4.39 |    4.39 |       0.455581 |                   2 |             4.39 |
| 2011-12-31 07:53:00 | nan    | nan    | nan    |  nan    |     nan        |                 nan |           nan    |
| 2011-12-31 07:54:00 | nan    | nan    | nan    |  nan    |     nan        |                 nan |           nan    |
| 2011-12-31 07:55:00 | nan    | nan    | nan    |  nan    |     nan        |                 nan |           nan    |
| 2011-12-31 07:56:00 | nan    | nan    | nan    |  nan    |     nan        |                 nan |           nan    |

### Data Dictonary
| Feature           | Datatype                         | Definition                                                 |
|:------------------|:---------------------------------|:-----------------------------------------------------------|
| Timestamp         | 4857377 non-null: datetime64[ns] | start tiem of time window (60s window), in Unix Time       |
| Open              | 3613769 non-null: float64        | Open price at start time window                            |
| High              | 3613769 non-null: float64        | High price within the time window                          |
| Low               | 3613769 non-null: float64        | Low price within the time window                           |
| Close             | 3613769 non-null: float64        | Close price at the end of the time window                  |
| Volume_(BTC)      | 3613769 non-null: float64        | Volume of BTC transacted in this window                    |
| Volume_(Currency) | 3613769 non-null: float64        | Volume of corresponding currency transacted in this window |
| Weighted_Price    | 3613769 non-null: float64        | VWAP - Volume Weighted Average Price         